http://setosa.io/ev/image-kernels/

# MNIST CNN

In [125]:
import tensorflow as tf

In [126]:
from tensorflow.examples.tutorials.mnist import input_data

In [127]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Helper Function

In [128]:
# Init weights

def init_weights(shape):
    
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    
    return tf.Variable(init_random_dist)

In [129]:
# Init biais

def init_biais(shape):
    
    init_biais_vals = tf.constant(0.1, shape=shape)
    
    return tf.Variable(init_biais_vals)

In [130]:
# Conv2D

def conv2d(x, W):
    
    # x => input tensor => [batch, H, W, Channels (color)]
    # W => kernel => [filter H, filter, W, Channels IN, Channels OUT]
    
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")

In [131]:
# Pooling layer

def max_pool_2by2(x):
    
    # x => input tensor => [batch, H, W, Channels]
    
    return tf.nn.max_pool(x, ksize=[1,2,2,1], 
                          strides=[1,2,2,1], padding="SAME")

In [132]:
# Convolutional Layer

def convolutional_layer(input_x, shape):
    
    W = init_weights(shape)
    b = init_biais([shape[3]])
    
    return tf.nn.relu(conv2d(input_x, W) + b)

In [133]:
# Normal Layer (Fully connected layer)

def normal_full_layer(input_layer, size):
    
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_biais([size])
    
    return tf.matmul(input_layer, W) + b

In [134]:
# Placeholders

x = tf.placeholder(tf.float32, shape=[None, 784])

y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [135]:
# Layers

# Reshape the flatten array into the image again
x_image = tf.reshape(x,[-1,28,28,1]) 

In [138]:
# First convolutional layer

# shape = [patchWidth, patchHeight, inputChannel(GrayScale), numberOfOutputChannel(features that you compute)]
convo_1 = convolutional_layer(x_image, shape=[5,5,1,32])

# Call my first pooling layer
convo_1_pooling = max_pool_2by2(convo_1)

In [140]:
# Second convolutional layer

# The input is now 32 and we want 64 features for each 5x5 patchs
convo_2 = convolutional_layer(convo_1_pooling, shape=[5,5,32,64])

# Call 2nd pooling layer
convo_2_pooling = max_pool_2by2(convo_2)

In [142]:
# Flatten out the result layer to connect to a fully connected layer

convo_2_flat = tf.reshape(convo_2_pooling, [-1,7*7*64])

full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

In [143]:
# Add Dropout

hold_prob = tf.placeholder(tf.float32)

full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

In [145]:
# Create our y_predictions

y_pred = normal_full_layer(full_one_dropout, 10)

In [146]:
# Loss Function

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [147]:
# Optimizer

optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
train = optimizer.minimize(cross_entropy)

In [148]:
# Initialization of the variables

init = tf.global_variables_initializer()

In [153]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x, batch_y = mnist.train.next_batch(50)
        
        sess.run(train, feed_dict={x:batch_x, y_true:batch_y, hold_prob:0.5})
        
        if i % 100 == 0:   # Every 100 steps
            
            print(f"On Step: {i}")
            print("Accuracy:")
            
            matches = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
            
            accuracy = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            print(sess.run(accuracy,feed_dict={x:mnist.test.images, y_true:mnist.test.labels, hold_prob:1.0}))
            print("\n")

On Step: 0
Accuracy:
0.1119


On Step: 100
Accuracy:
0.9489


On Step: 200
Accuracy:
0.9652


On Step: 300
Accuracy:
0.9706


On Step: 400
Accuracy:
0.9745


On Step: 500
Accuracy:
0.9772


On Step: 600
Accuracy:
0.9799


On Step: 700
Accuracy:
0.982


On Step: 800
Accuracy:
0.9822


On Step: 900
Accuracy:
0.9858


On Step: 1000
Accuracy:
0.9869


On Step: 1100
Accuracy:
0.9846


On Step: 1200
Accuracy:
0.9865


On Step: 1300
Accuracy:
0.9864


On Step: 1400
Accuracy:
0.9845


On Step: 1500
Accuracy:
0.9871


On Step: 1600
Accuracy:
0.9854


On Step: 1700
Accuracy:
0.9872


On Step: 1800
Accuracy:
0.9878


On Step: 1900
Accuracy:
0.9842


On Step: 2000
Accuracy:
0.9867


On Step: 2100
Accuracy:
0.9891


On Step: 2200
Accuracy:
0.9867


On Step: 2300
Accuracy:
0.9882


On Step: 2400
Accuracy:
0.9897


On Step: 2500
Accuracy:
0.9903


On Step: 2600
Accuracy:
0.9852


On Step: 2700
Accuracy:
0.9866


On Step: 2800
Accuracy:
0.9873


On Step: 2900
Accuracy:
0.9882


On Step: 3000
Accuracy: